In [13]:
import pennylane as qml
from pennylane import numpy as np
import nlopt #pip install nlopt
import random


import warnings
#warnings.filterwarnings('ignore')


## Create Quantum Optical Device

In [14]:
dev = qml.device("strawberryfields.fock", wires=4, cutoff_dim=4)

In [15]:
def layer(theta, phi, wires):
    M = len(wires)
    phi_nonlinear = np.pi / 2

    qml.templates.Interferometer(
        theta, phi, np.zeros(M), wires=wires, mesh="triangular",
    )

    for i in wires:
        qml.Kerr(phi_nonlinear, wires=i)

In [16]:
@qml.qnode(dev)
def quantum_neural_net(var, x):
    wires = list(range(len(x)))

    # Encode input x into a sequence of quantum fock states
    for i in wires:
        qml.FockState(x[i], wires=i)

    # "layer" subcircuits
    for i, v in enumerate(var):
        layer(v[: len(v) // 2], v[len(v) // 2 :], wires)

    return [qml.expval(qml.NumberOperator(w)) for w in wires]

## Cost Function

In [17]:
def square_loss(labels, predictions):
    term = 0
    for l, p in zip(labels, predictions):
        lnorm = l / np.linalg.norm(l)
        pnorm = p / np.linalg.norm(p)

        term = term + np.abs(np.dot(lnorm, pnorm.T)) ** 2

    return 1 - term / len(labels)

In [18]:
def cost(var, data_input, labels):
    predictions = np.array([quantum_neural_net(var, x) for x in data_input])
    sl = square_loss(labels, predictions)

    return sl

## Generic Gate we are training network to emulate

In [19]:
# Define the CNOT input-output states (dual-rail encoding) and initialize
# them as non-differentiable.

X = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 1, 0],
              [0, 1, 0, 1]], requires_grad=False)

Y = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 0, 1],
              [0, 1, 1, 0]], requires_grad=False)

## Quantum Optical Machine Learning

In [ ]:
"""
Goal is to create a CNOT gate in an optical device (photonic devices dont have native cnot gates) -> X, P, V, K
"""

In [20]:
num_layers = 2
M = len(X[0])
num_variables_per_layer = M * (M - 1)

var_init = (4 * np.random.rand(num_layers, num_variables_per_layer) - 2) * np.pi
print(var_init)

[[-6.1943875  -5.99337433  0.31132756 -1.25838414 -5.69676729  5.95338743
  -3.35809437 -5.14459127  1.48768247 -1.47702618  6.0724584  -0.4176698 ]
 [ 4.52314455  2.26581135 -0.62204475 -6.11649289  5.55687115  0.7953032
  -1.4398987  -6.08254746 -3.38168792 -3.25436997  2.3029573   1.38225877]]


In [21]:
cost_grad = qml.grad(cost)

print_every = 1

# Wrap the cost so that NLopt can use it for gradient-based optimizations
evals = 0
def cost_wrapper(var, grad=[]):
    global evals
    evals += 1

    if grad.size > 0:
        # Get the gradient for `var` by first "unflattening" it
        var_grad = cost_grad(var.reshape((num_layers, num_variables_per_layer)), X, Y)
        grad[:] = var_grad.flatten()
    cost_val = cost(var.reshape((num_layers, num_variables_per_layer)), X, Y)

    if evals % print_every == 0:
        print(f"Iter: {evals:4d}    Cost: {cost_val:.4e}")

    return float(cost_val)


# Choose an algorithm
opt_algorithm = nlopt.LD_LBFGS  # Gradient-based
# opt_algorithm = nlopt.LN_BOBYQA  # Gradient-free

opt = nlopt.opt(opt_algorithm, num_layers*num_variables_per_layer)

opt.set_min_objective(cost_wrapper)

opt.set_lower_bounds(-2*np.pi * np.ones(num_layers*num_variables_per_layer))
opt.set_upper_bounds(2*np.pi * np.ones(num_layers*num_variables_per_layer))

var = opt.optimize(var_init.flatten())
var = var.reshape(var_init.shape)

Iter:    1    Cost: 4.0045e-01
Iter:    2    Cost: 2.3274e-01
Iter:    3    Cost: 2.2149e-01
Iter:    4    Cost: 1.1658e-01
Iter:    5    Cost: 1.1556e-01
Iter:    6    Cost: 5.8124e-02
Iter:    7    Cost: 4.0028e-02
Iter:    8    Cost: 2.5696e-02
Iter:    9    Cost: 7.1356e-03
Iter:   10    Cost: 2.9986e-03
Iter:   11    Cost: 1.2262e-03
Iter:   12    Cost: 7.5319e-04
Iter:   13    Cost: 4.2233e-04
Iter:   14    Cost: 2.4666e-04
Iter:   15    Cost: 2.3732e-04
Iter:   16    Cost: 7.5397e-05
Iter:   17    Cost: 5.1838e-05
Iter:   18    Cost: 1.9697e-05
Iter:   19    Cost: 6.7224e-06
Iter:   20    Cost: 2.1031e-06
Iter:   21    Cost: 7.0350e-07
Iter:   22    Cost: 1.6842e-07
Iter:   23    Cost: 6.0320e-08
Iter:   24    Cost: 2.0190e-08
Iter:   25    Cost: 6.6324e-09
Iter:   26    Cost: 2.1623e-09
Iter:   27    Cost: 6.9723e-10
Iter:   28    Cost: 2.2163e-10
Iter:   29    Cost: 6.9445e-11
Iter:   30    Cost: 2.2087e-11


In [22]:
print(f"The optimized parameters (layers, parameters):\n {var}\n")

Y_pred = np.array([quantum_neural_net(var, x) for x in X])
for i, x in enumerate(X):
    print(f"{x} --> {Y_pred[i].round(2)}, should be {Y[i]}")

The optimized parameters (layers, parameters):
 [[-6.27119279e+00 -6.28267943e+00 -5.01836140e-05 -7.89804090e-01
  -5.49762742e+00  6.28318531e+00 -3.37093158e+00 -5.18261959e+00
   1.44555920e+00 -2.16148584e+00  6.14482778e+00 -6.82043927e-02]
 [ 3.14282021e+00  2.45909763e+00 -2.56092554e-03 -6.28318531e+00
   4.83512685e+00  7.84675767e-01 -1.27958631e+00 -6.28318531e+00
  -3.33956475e+00 -3.18318860e+00  2.56858860e+00  1.49859585e+00]]

[1 0 1 0] --> [1. 0. 1. 0.], should be [1 0 1 0]
[1 0 0 1] --> [1. 0. 0. 1.], should be [1 0 0 1]
[0 1 1 0] --> [0. 1. 0. 1.], should be [0 1 0 1]
[0 1 0 1] --> [0. 1. 1. 0.], should be [0 1 1 0]


In [24]:
quantum_neural_net(var_init, X[0])

tensor([0.61007575, 0.64204904, 0.64257627, 0.10529894], requires_grad=True)

In [28]:
print(quantum_neural_net.draw())

 0: ──|1⟩────────────────────────────────────────╭BS(0.311, 1.49)────R(0)─────────────Kerr(1.57)────────────────────────────────────────────────────────────────────╭BS(-0.622, -3.38)───R(0)────────────Kerr(1.57)───────────────────────────────┤ ⟨n⟩ 
 1: ──|0⟩─────────────────────╭BS(-5.99, -5.14)──╰BS(0.311, 1.49)───╭BS(-5.7, 6.07)───R(0)──────────────Kerr(1.57)────────────────────────────────╭BS(2.27, -6.08)──╰BS(-0.622, -3.38)──╭BS(5.56, 2.3)───R(0)─────────────Kerr(1.57)──────────────┤ ⟨n⟩ 
 2: ──|1⟩──╭BS(-6.19, -3.36)──╰BS(-5.99, -5.14)──╭BS(-1.26, -1.48)──╰BS(-5.7, 6.07)──╭BS(5.95, -0.418)──R(0)────────Kerr(1.57)──╭BS(4.52, -1.44)──╰BS(2.27, -6.08)──╭BS(-6.12, -3.25)───╰BS(5.56, 2.3)──╭BS(0.795, 1.38)──R(0)────────Kerr(1.57)──┤ ⟨n⟩ 
 3: ──|0⟩──╰BS(-6.19, -3.36)─────────────────────╰BS(-1.26, -1.48)───────────────────╰BS(5.95, -0.418)──R(0)────────Kerr(1.57)──╰BS(4.52, -1.44)────────────────────╰BS(-6.12, -3.25)───────────────────╰BS(0.795, 1.38)──R(0)────────Kerr(1.57)──┤ ⟨n⟩ 

